In [1]:
##
import sys
import os
import pickle
import sys
import warnings
import enum
from enum import Enum
import concurrent
from typing import Optional, List, Dict, Callable

##
import matplotlib
from pathlib import Path

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER

In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [3]:
##
import pandas as pd
import tqdm
from dotenv import load_dotenv
import financedatabase as fd
import fundamentalanalysis as fa

##
from common.utils import now_time
from configuration.settings import ProjectDir
from rl.data.financial_modeling_prep import download
from rl.data.Company import Company

In [4]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
load_dotenv(prj_dir.root.joinpath("env/.env"))

_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"
tickers = DOW_30_TICKER

In [5]:
try:
    print(os.getenv("EOD_HISTORICAL_DATA_API"))
    print(os.getenv("NASDAQ_HISTORICAL_DATA_API"))
    print(os.getenv("ALPHA_VANTAGE_API"))
    print(os.getenv("FINANCIAL_MODELING_PREP_API"))
except:
    raise ValueError("ERROR: API keys")

639b4cc5af44e7.93519913
B9p3RKb-K6xWwBRLso7Z
G21Y7NB8U8VJFFIH
6b0006554aa86e3e21493713af2af3c4


In [6]:
def get_download_range(_from: int, _plus_costant: int) -> tuple:
    return (_from, _from + _plus_costant - 1)


def download_subprocess(start: int, stop: int, step: int) -> dict:
    all_symbols_info = dict()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        result = [
            executor.submit(
                download, companies.index[get_download_range(i, step)[0] : get_download_range(i, step)[1]], api_key
            )
            for i in range(start, stop, step)
        ]

        for i in concurrent.futures.as_completed(result):
            try:
                all_symbols_info.update(i.result())
            except Exception as e:
                print(e)
    return all_symbols_info

In [7]:
#
ticker = "MSFT"
api_key = os.getenv("FINANCIAL_MODELING_PREP_API")

# Show the available companies
companies = fa.available_companies(api_key)

In [8]:
print(f"{companies.shape=}")

companies.shape=(67363, 5)


In [9]:
#
start = 6200
stop = companies.index.size
step = 200
processes = 4

In [ ]:
for i in range(start, stop, step):
    ##
    _start = i
    _stop = i + step - 1
    _step = step // processes

    # get_download_range()
    print([get_download_range(i, _step) for i in range(_start, _stop, _step)])
    # continue

    ##
    filename = prj_dir.dataset.financial_modeling_prep.joinpath(f"{_start}_{_stop}_companies_{now_time()}.pkl")
    comps: dict = download_subprocess(_start, _stop, _step)
    comps = download_subprocess(_start, _stop, _step)
    with open(file=filename, mode="wb") as f:
        import pickle

        pickle.dump(comps, f)

[(6200, 6249), (6250, 6299), (6300, 6349), (6350, 6399)]


Symbol: PRTK:  43%|████▎     | 21/49 [01:45<02:13,  4.78s/it] ]        

In [ ]:
filename = prj_dir.dataset.financial_modeling_prep.joinpath(f"6000_6199_companies_2022-12-17T02-36-00.pkl")
with open(file=filename, mode="rb") as f:
    import pickle

    c: Dict[str, Company] = pickle.load(f)

In [17]:
c.keys().__len__()

196